In [2]:
!pip install -e ../../gym_k8s_real

Obtaining file:///Users/zxp/Desktop/learn/0_Project/EPI/EPI_kube_scaling/HEURIST-MAL-k8s/gym_k8s_real
  Preparing metadata (setup.py) ... done
  Running setup.py develop for gym-k8s-real


In [3]:
import gym
import random
import subprocess
import time
import numpy as np
from threading import Lock, Thread
import datetime
import gym_k8s_real

ModuleNotFoundError: No module named 'gym_k8s_real'

## Info about the kubernetes environment we deploy

In [2]:
# Timestep duration in minutes
# We wait these many minutes for our actions to be enforced
timestep_duration = 1.5
app_name = 'firewall'
memory_req = '128Mi'
cpu_req = '80m'
sla_latency = 2.6
sla_host = 'http://145.100.135.89:6088/'
# latency metric
sla_metric_name = 'latency'
gym_env = 'gym_k8s_real:k8s-env-discrete-state-discrete-action-v2'
#init Q_table
q_table_file = 'Q-env-discrete-state-discrete-action-data.npy'
q_table_init_value = 12.5
total_epochs = 100
num_of_services = 1
steps_per_epoch = 15

## Load or create the Q table
If our Q table file is present, we load in into memory. Otherwise we create a gym environment and using the environment's observation and space dimensions, we create a new one.

In [8]:
try:
    q_table = np.load(q_table_file)
    print('File present. Loading done!')
    
except IOError:
    env = gym.make(
        gym_env,
        timestep_duration=timestep_duration,
        app_name=app_name,
        sla_latency=sla_throughput,
        sla_host=sla_host,
        sla_latency_metric_name=sla_latency_metric_name
    )

    q_table = np.full((env.observation_space.n, env.action_space.n), q_table_init_value)
    np.save(q_table_file, q_table)

    print('File not present. Created successfully!')

table_lock = Lock()

File present. Loading done!


In [9]:
(7 ** 9+ 6 ** 3+3 ** 6)*21

847445592

In [ ]:
q_table = np.full((882, 882, 882, 7, 21), 1)

In [23]:
(7*7*6*3)**3*21

14408708328

In [26]:
(882**3)*7*21

100860958296

## Create historical states csv file if it doesn't exist

In [4]:
try:
    open('k8s_historical_states_discrete.csv', 'r').close()
    print('File already present.')
except IOError:
    with open('k8s_historical_states_discrete.csv', 'w') as f:
        f.write('current_app_name,timestep,state,action,next_state,reward,'
                'done,number_of_pods,cpu_util,latency_violation,latency,hpa_threshold,info\n')
    print('File not present. Created successfully!')

File not present. Created successfully!


## Define function to calculate heuristic value

In [ ]:
#calculate heuristic value; input: action from heuristic policy
#hAction is returned from gym environment and is encoded 
def hValue(state, hAction):
    #Q-value where action is equal to heuristic action
    heurQ = q_table[state][hAction]
    hVal = np.max(q_table[state]) 
    return hVal - heurQ

# Train the agent

## Agent training
This function trains our agent:

In [5]:
def decode(i):
        out = []
        out.append(i % 7)
        i //= 7
        out.append(i % 5)
        i //= 5
        out.append(i % 5)
        i //= 5
        out.append(i)
        return reversed(out)

In [12]:
def train_agent(num_service):
    # Hyperparameters
    alpha = 0.1
    gamma = 0.9
    epsilon_init = 0.97
    epsilon_min = 0.2

    current_app_name = app_name
    current_throughput_metric_name = prometheus_throughput_metric_name
    
    env = gym.make(
        gym_env,
        timestep_duration=timestep_duration,
        app_name=current_app_name,
        sla_throughput=sla_throughput,
        prometheus_host=prometheus_host,
        prometheus_throughput_metric_name=current_throughput_metric_name
    )

    for epoch in range(0, total_epochs):
        state, _, hAction = env.reset()
#         state, _ = env.reset()
        decoded_state = list(decode(state))
        print('======EPOCH{}=======\n'
              'Training started with cpu utilization: {}, hpa cpu threshold: {}, number of pods: {}, latency:{}'
             .format(epoch, decoded_state[0], decoded_state[1], decoded_state[2], decoded_state[3]))
     
        done = False

        for step in range(steps_per_epoch):
            current_timestep = epoch * steps_per_epoch + step
            q_table = np.load(q_table_file)
            
            # Epsilon keeps getting smaller and stops when it reaches epsilon_min
            current_epsilon = pow(epsilon_init, current_timestep)
            epsilon = max(current_epsilon, epsilon_min)
            
            if done:
                break
            else: 
                # Explore action space, non-greedy (NG) action selection
                if random.uniform(0, 1) < epsilon:
                    action = env.action_space.sample()  
                # Exploit learned values, greedy (G) action selection
                else:
                    hVal = hValue(state, hAction)
                    maxAction = -1
                    maxQ = -1
                    for action in q_table[state]:
                        aggreVal = q_table[state][action] + hVal if action == hAction else q_table[state][action] + 0
                        if maxQ < aggreVal:
                            maxQ = aggreVal
                            maxAction = action
                
#                 decoded_state = list(decode(state))
#                 print('======ROUND{}=======\n'
#                       'app: {}, pod_cpu_util: {}, cpu_threshold: {}, number_of_pods: {}, latency: {}, latency_violation: {}, action: {}'
#                       .format(step, current_app_name, decoded_state[0], decoded_state[1], decoded_state[2], 
#                               decoded_state[3], int(decoded_state[3] >= 5), action))
                
                real_ob, reward, done, next_state, hAction = env.step(action)
#                 real_ob, reward, done, next_state = env.step(action)
                
                now = datetime.datetime.now() + datetime.timedelta(hours=2)
                dt_string = now.strftime('%d/%m/%Y %H:%M:%S')
                dt_dict = {
                    'datetime': dt_string
                }
                info = dt_dict
                
                #To-do: change real_ob
                (pod_cpu_util,
                 cpu_threshold,
                 number_of_pods,
                 latency) = real_ob

                # Latency violation becomes 1 if the SLA was violated
                # otherwise it's 0
                latency_violation = int(latency > sla_latency)
                

                # Save historical tuple
                with open('k8s_historical_states_discrete.csv', 'a') as f:
                    f.write(
                        '{},{},{},{},'.format(current_app_name, current_timestep, state, action) +
                        '{},{},{},{},'.format(next_state, reward, done, number_of_pods) +
                        '{},{},{},{},{}'.format(pod_cpu_util, throughput_violation, throughput, cpu_threshold, info) +
                        '\n'
                    )
                    
                #Update Q-value
                with table_lock:
                    old_value = q_table[state, action]
                    # Q-table update is always greedy (np.max)
                    # Q-learning is off-policy since the action taken can be of a different policy (non-greedy, random) (NG)
                    next_max = np.max(q_table[next_state])
                    new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
                    q_table[state, action] = new_value
                    
                #To-do: Update H-value
                
                
                print('reward: {}, new_value: {}'.format(reward, new_value))
                
#                 print('new cpu utilization: {}, new hpa cpu threshold: {}, new number of pods: {}, new latency: {}, new latency_violation: {}'
#                       .format(pod_cpu_util, cpu_threshold, number_of_pods, throughput, throughput_violation)
#                       .format(decoded_state[0], decoded_state[1], decoded_state[2], decoded_state[3]))
                
                state = next_state
                
                with table_lock:
                    np.save(q_table_file, q_table)

        with table_lock:
            np.save(q_table_file, q_table)

        print('Training finished.\n')

In [ ]:
train_agent(num_of_services)